# Twitter API

## Tweepy

### https://developer.twitter.com/apps

In [ ]:
# -*- coding: utf-8 -*-

import tweepy
from tweepy.api import API
import pandas as pd
Token_API = [" "," "," "," "]
ACCESS_TOKEN = Token_API[0]
ACCESS_TOKEN_SECRET = Token_API[1]
API_KEY = Token_API[2]
API_SECRET = Token_API[3]

In [ ]:
key = tweepy.OAuthHandler(API_KEY, API_SECRET)
key.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(key)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

### Twitter User

In [ ]:

user = api.get_user("M0htat")
user


In [ ]:
print("User details:")
print(user.id)
print(user.name)
print(user.description)
print(user.location)
print(user.created_at)
print(user.friends_count)
print(user.geo_enabled)
print(user.lang)
print(dict(user.entities)['url']['urls'][0]['expanded_url'])
print(user.followers_count)

In [ ]:
print("\nLast 20 Followers:")
for follower in user.followers():
    print(follower.name)

In [ ]:
import time

ids = []
for page in tweepy.Cursor(api.followers_ids, screen_name="M0htat").pages():
    ids.extend(page)
    time.sleep(5)

print(len(ids))

ids

In [ ]:
TextsUser = api.user_timeline(screen_name = 'M0htat', count = 1000, include_rts = True)

TextsUser

In [ ]:
users_locs = [[status.text, status.created_at,status.lang,status.favorite_count,status.retweet_count]for status in TextsUser]

tweet_User = pd.DataFrame(data=users_locs, 
                    columns=['Text', "Date","Lang","favorite_count","retweet_count"])
tweet_User

### Twiteer Search

In [ ]:
for tweet in tweepy.Cursor(api.search, q='#فیروزآبادی',result_type='popular').items(20):
    print(tweet.text)

In [ ]:
date_since = "2021-09-01"
tweets = tweepy.Cursor(api.search,
              q='#رونالدو',
              lang="fa",
              since=date_since).items(1000)
xxx = 0
for tweet in tweets:
    print(xxx,"\n",tweet.text,"<>",tweet.user.screen_name,tweet.created_at)
    xxx +=1

In [ ]:
import pandas as pd
tweets = tweepy.Cursor(api.search, 
                           q='رونالدو',
                           lang="fa",
                           since=date_since).items(100)

users_locs = [[tweet.user.screen_name, tweet.user.location,tweet.text,tweet.created_at] for tweet in tweets]

tweet_text = pd.DataFrame(data=users_locs, 
                    columns=['user', "location","Text","Time"])
tweet_text

### Twitter Trends

In [ ]:
api.trends_available()

In [ ]:
trends_result = api.trends_place(1)
for trend in trends_result[0]["trends"]:
    print(trend["name"],trend["tweet_volume"])
    
#api.trends_available()

## Twitter Streaming 

In [ ]:
import simplejson as json
import codecs

class listener(tweepy.StreamListener):
   def on_data(self, data):
      try:
         all_data = json.loads(data)
         tweet = all_data['extended_tweet']['full_text']
         encoded = tweet.encode("utf-8", errors='ignore').decode('UTF-8')
         username = all_data["user"]["screen_name"]
         print (username," :: ", encoded,"\n"*3)
            
      except:
         all_data = json.loads(data)
         tweet = all_data["text"]
         encoded = tweet.encode("utf-8", errors='ignore').decode('UTF-8')
         username = all_data["user"]["screen_name"]
         print (username," :: ", encoded,"\n"*3)
        
      return True
   def on_error(self, status):
      print (status)


keywords = ['رونالدو']
twitterStream = tweepy.Stream(key, listener(),tweet_mode='extended')
try:
   twitterStream.filter(track=keywords)
finally:
   pass